# Chapter 2, Section 2.22 Creating a Dockerfile for the image

## Objectives
1. Build Node script with Dockerfile (Node base image) 
2. Push to DockerHub
3. Pull from DockerHub

---

## Setup

Create Node script as `app.js`

Use prepared script from GitHub repo for *[kubernetes-in-action-2nd-edition](https://github.com/luksa/kubernetes-in-action-2nd-edition/blob/master/Chapter02/kiada-0.1/app.js)*


```javascript
const http = require('http');
const os = require('os');
const fs = require('fs');

const version = "0.1";
const listenPort = 8080;

function sendResponse(status, contentType, encoding, body, response) {
    response.writeHead(status, {'Content-Type': contentType});
    response.write(body, encoding);
    response.end();
}

function renderFile(req, res, path, contentType) {
    let template = fs.readFileSync(path, 'utf8');

    let map = Object.assign({
        "{{hostname}}": os.hostname(),
        "{{clientIP}}": req.connection.remoteAddress,
        "{{version}}": version,
    });

    let body = template.replace(
        new RegExp(Object.keys(map).join('|'), 'g'),
        function (matched) {
            return map[matched];
        });

    sendResponse(200, contentType, 'utf8', body, res);
}

function sendFile(req, res, path, contentType) {
    let body = fs.readFileSync(path, 'binary');
    sendResponse(200, contentType, 'binary', body, res);
}

// this function guesses if the client that sent the request is a full-fledged
// graphical web browser and not a text-based tool such as curl
// graphical browsers typically send accept: text/html,application/xhtml+xml,...
// curl sends accept: */*
function isGraphicalWebBrowser(req) {
    let accept = req.headers.accept || "*/*";
    return accept.startsWith("text/html");
}

function handler(req, res) {
    let clientIP = req.connection.remoteAddress;
    console.log("Received request for " + req.url + " from " + clientIP);
    switch (req.url) {
        case '/':
            if (isGraphicalWebBrowser(req)) {
                res.writeHead(302, {"Location": "html"});
                res.write("Redirecting to the html version...");
                res.end();
                return;
            }
        // text-based clients fall through to the '/text' case
        case '/text':
            return renderFile(req, res, "html/index.txt", "text/plain");
        case '/html':
            return renderFile(req, res, "html/index.html", "text/html");
        case '/stylesheet.css':
            return sendFile(req, res, "html/stylesheet.css", "text/css");
        case '/javascript.js':
            return sendFile(req, res, "html/javascript.js", "text/javascript");
        case '/favicon.ico':
            return sendFile(req, res, "html/favicon.ico", "image/x-icon");
        case '/cover.png':
            return sendFile(req, res, "html/cover.png", "image/png");
        default:
            return sendResponse(404, "text/plain", "utf8", req.url + " not found", res)
    }
}

console.log("Kiada - Kubernetes in Action Demo Application");
console.log("---------------------------------------------");
console.log("Kiada " + version + " starting...");
console.log("Local hostname is " + os.hostname());
console.log("Listening on port " + listenPort);

let server = http.createServer(handler);
server.listen(listenPort);

```

---

Create `Dockerfile`

```bash
FROM node:16
COPY app.js /app.js
COPY html/ /html
ENTRYPOINT ["node", "app.js"]
```




---

## Docker operations

Build Docker image and tag as `kiada`

We use --quiet here since the output is very noisy in a Jupyter notebook (compared to the CLI).

In [4]:
!docker build -t kiada ./src --quiet
print("Done")

sha256:3c124ed8dcabe2e9677708fe81e59ad4876e9ba69625759dc30b94a8c3d65c37
Done


---
Check images

In [5]:
!docker images

REPOSITORY   TAG       IMAGE ID       CREATED          SIZE
kiada        latest    3c124ed8dcab   12 seconds ago   911MB


---
Create new container named `kiada-container` from the `kiada` image. 

The container will be detached from the local console via the -d flag.

Port 8080 on the local machine will be mapped to port 8080 inside the container.


In [6]:
!docker run --name kiada-container -p 8080:8080 -d kiada

be3d95b146e633bf99086824ab83bd5e788c1f4777df7c1ae63b762040c75546


---
Verify this new container process is running.


In [7]:
!docker ps

CONTAINER ID   IMAGE     COMMAND         CREATED         STATUS         PORTS                    NAMES
be3d95b146e6   kiada     "node app.js"   4 seconds ago   Up 3 seconds   0.0.0.0:8080->8080/tcp   kiada-container


---
Test via curl (inside the multipass instance)


In [8]:
!curl localhost:8080

Kiada version 0.1. Request processed by "be3d95b146e6". Client IP: ::ffff:172.17.0.1


---
If applicable, run `multipass info actionbook-v2` to find the external IP, and use that for browser access. For example, `http://192.168.64.5:8080/`

---
Explore different Docker operations.

In [9]:
!docker ps

CONTAINER ID   IMAGE     COMMAND         CREATED          STATUS          PORTS                    NAMES
be3d95b146e6   kiada     "node app.js"   11 seconds ago   Up 11 seconds   0.0.0.0:8080->8080/tcp   kiada-container


In [10]:
!docker stop kiada-container
print("Done")

kiada-container
Done


In [11]:
!docker start kiada-container
print("Done")

kiada-container
Done


In [12]:
!docker stop kiada-container
!docker rm kiada-container
print("Done")

kiada-container
kiada-container
Done


In [13]:
!docker rmi kiada
print("Done")

Untagged: kiada:latest
Deleted: sha256:3c124ed8dcabe2e9677708fe81e59ad4876e9ba69625759dc30b94a8c3d65c37
Done


In [14]:
!docker images prune
print("Done")

REPOSITORY   TAG       IMAGE ID   CREATED   SIZE
Done


---
# Create another image, then push to DockerHub

In [15]:
!docker build -t kiada ./src --quiet

sha256:3c124ed8dcabe2e9677708fe81e59ad4876e9ba69625759dc30b94a8c3d65c37


---
Add an additional tag for the image. The current name is kiada. We want to tag it as

```bash
<username>/kiada:<version>
``` 

       
    

In [16]:
!docker tag kiada georgebaptista/kiada:0.1
print("Done")

Done


In [17]:
!docker images

REPOSITORY             TAG       IMAGE ID       CREATED              SIZE
georgebaptista/kiada   0.1       3c124ed8dcab   About a minute ago   911MB
kiada                  latest    3c124ed8dcab   About a minute ago   911MB


---
If initial login to DockerHub, need to login with username/password, eg
```
docker login --username ..... --password ...
```

In [45]:
# !docker login --username $USER --password $PASSWORD

In [18]:
!docker push georgebaptista/kiada:0.1

The push refers to repository [docker.io/georgebaptista/kiada]

5707de35: Preparing 
da07641c: Preparing 
ec3f9823: Preparing 
9b05cfc9: Preparing 
aa46bd11: Preparing 
ef705308: Preparing 
ee259bc5: Preparing 
fe198f83: Preparing 
c4b93150: Preparing 
3d4fb826: Preparing 
707de35: Pushed lready exists 1kB0.1: digest: sha256:2d86a1a0459db9944192d87769729b09ae0e005dac338a19215892c679aa19e2 size: 2632


---
## Clean-up (if necessary)

In [19]:
!docker images

REPOSITORY             TAG       IMAGE ID       CREATED              SIZE
kiada                  latest    3c124ed8dcab   About a minute ago   911MB
georgebaptista/kiada   0.1       3c124ed8dcab   About a minute ago   911MB


In [22]:
# Stop all containers
!docker stop $(docker ps -a -q)

"docker stop" requires at least 1 argument.
See 'docker stop --help'.

Usage:  docker stop [OPTIONS] CONTAINER [CONTAINER...]

Stop one or more running containers


In [23]:
# Remove all containers
!docker rm $(docker ps -a -q)

"docker rm" requires at least 1 argument.
See 'docker rm --help'.

Usage:  docker rm [OPTIONS] CONTAINER [CONTAINER...]

Remove one or more containers


In [24]:
# Remove all images
!docker rmi $(docker images -a -q) --force

Untagged: georgebaptista/kiada:0.1
Untagged: georgebaptista/kiada@sha256:2d86a1a0459db9944192d87769729b09ae0e005dac338a19215892c679aa19e2
Untagged: kiada:latest
Deleted: sha256:3c124ed8dcabe2e9677708fe81e59ad4876e9ba69625759dc30b94a8c3d65c37
Error: No such image: 3c124ed8dcab


In [25]:
# Clean-up
!docker system prune --force

Deleted build cache objects:
4gajmsy8prsdxneacgefgrbx6
89guezulfmk54drgchcpwlrv8
278xp3qyp549x8d7lo0llhdiz
otryj9eue2x97jx9hxdm9n1rk
zibrm19ws7o4guw7o2aqghys1
psd6d7duo0pnoj7olx768r7kj
tf3p2zx43u52ig887br7ajfa0
id5jifvlpc203jmf72kmoz1w4
gvu55b5hlnfigl4wjled47oob
ono3ollnj2n9x7fm52543u43r
5jpvsw0iny86rss2cwdadye5p
yhqilghums4lht3frn9zo4cpk
mnwwiwcqak2dl3dkgrnm6cg01
pqs9gsz4wp8l4cnknn0t8zhj0

Total reclaimed space: 1.072MB
